In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [47]:
# run data_processing.ipynb to generate the test set here
TEST_DATA = 'data/test.80000.csv'
df_test = pd.read_csv(TEST_DATA)
df_test.head()

,text,target
0,USERNAME yeah sure whatevs...haha you have to ...,1
1,USERNAME but you should always eat breakfast! ...,1
2,URL - apple bathrobe i got from friends for my...,1
3,where did the sun go? typical shitty british w...,0
4,cozy day at home,1


## Baseline

Use some naive model or cite previous paper? 
1. use some naive model - like sentiment lexicon and classify positive if most words are positive
- pros: can replicate
2. use previous research results
- concerns: may not be able to find original source code to replicate baseline results

#### TwitrRatr - same baseline as https://www-cs.stanford.edu/people/alecmgo/papers/TwitterDistantSupervision09.pdf

In [2]:
# TODO : CANNOT FIND LIST OF POSITIVE/NEGATIVE WORDS MENTIONED IN THE PAPER???

#### Opinion lexicon - https://github.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/blob/master/data/opinion-lexicon-English/negative-words.txt

In [45]:
POSITIVE_FILE = 'data/positive-words.txt'
positive_words = pd.read_table(POSITIVE_FILE, skiprows=34, names=['words'], encoding='ISO-8859-1')['words'].tolist()
positive_words[:10]

['a+',
 'abound',
 'abounds',
 'abundance',
 'abundant',
 'accessable',
 'accessible',
 'acclaim',
 'acclaimed',
 'acclamation']

In [46]:
NEGATIVE_FILE = 'data/negative-words.txt'
negative_words = pd.read_table(NEGATIVE_FILE, skiprows=34, names=['words'], encoding='ISO-8859-1')['words'].tolist()
negative_words[:10]

['2-faced',
 '2-faces',
 'abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted']

In [50]:
# split based on all punctuations possible
def split_tweet(tweet):
    return "".join((char if char.isalpha() else " ") for char in tweet).split()

def count_pos(tweet):
    tweet_words = split_tweet(tweet)
    return sum([1 for w in tweet_words if w in positive_words])

def count_neg(tweet):
    tweet_words = split_tweet(tweet)
    return sum([1 for w in tweet_words if w in negative_words])

In [52]:
df_test.text[0]

'USERNAME yeah sure whatevs...haha you have to admit the sweet and innocent are by far the best and a epic win '

In [49]:
count_pos(df_test.text[0])

3

In [51]:
count_neg(df_test.text[0])

0

In [54]:
pred = df_test.text.apply(lambda x: 1 if count_pos(x) - count_neg(x) >= 0 else 0)

In [58]:
pred

0        1
1        1
2        1
3        1
4        1
        ..
79995    1
79996    1
79997    1
79998    1
79999    0
Name: text, Length: 80000, dtype: int64

## Evaluation Metric

TODO: What to pick for evaluation metric?

#### F1-score and Confusion Matrix

In [59]:
from sklearn.metrics import confusion_matrix
confusion_matrix(df_test.target, pred)

array([[14693, 25453],
       [ 3985, 35869]], dtype=int64)

In [61]:
from sklearn.metrics import f1_score
f1_score(df_test.target, pred, average='macro')

0.6042998367486137